In [55]:
# Quick hello world
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-pro")
llm.invoke("What is the capital of France")

In [ ]:
# Chat prompt
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are now an English teacher who helps students learn English through news articles. First, student's preferred learning level will be given as an initial input. Then you should ask a topic. Based on this level and topic information, find a relevant news article from the past week on the chosen topic. The learning process will be divided into three main steps. First, extract key vocabulary from the article and check if the student is familiar with their meanings. Second, present the article to the student. Finally, test the student on essential grammar points and their understanding of the content based on the article.Only check student's level once."),
    ]
)

# 3. define chain
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser()
chain.invoke({})

In [57]:
import gradio as gr

from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage()

In [41]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [ ]:
# More general chat
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are now an English teacher who helps students learn English through news articles. Learning level will be given as an initial input. First, you need to gather information on the student's preferred topic. Based on this information, find a relevant news article from the past week on the chosen topic. The learning process will be divided into three main steps. First, extract key vocabulary from the article and check if the student is familiar with their meanings. Second, present the article to the student. Finally, test the student on essential grammar points and their understanding of the content based on the article."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)
chain = chat_with_history_prompt | llm | StrOutputParser()

def chat(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    generator = chain.stream({"message": message, "history": history_langchain_format})

    assistant = ""
    for gen in generator:
        assistant += gen
        yield assistant

In [76]:
def chat(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    return chain.invoke({"message": message, "history": history_langchain_format})

In [77]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "Beginner",
            "Intermidiate",
            "Advanced",
        ],
        title="Solar Chatbot",
        description="Upstage Solar Chatbot",
    )
    chatbot.chatbot.height = 300

In [78]:
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
